In [ ]:
# create a list of dictionaries
dict_container = []
for file_path, features in data.items():
    # convert any float values in features to a list
    features = [features] if isinstance(features, float) else features
    
    # create a dictionary for each file_path and features
    feature_dict = {"file_path": file_path}
    feature_dict.update({f"feature_{i}": float(val) for i, val in enumerate(features)})
    dict_container.append(feature_dict)

# create a DataFrame
df = pd.DataFrame(dict_container)

In [1]:
import os
import sys
import pandas as pd
import pickle
from pathlib import Path

# Setting some paths
data_root = Path('data').resolve()
audio_root = Path('audio').resolve()
essentia_descriptors = Path(data_root / 'ext_desc_output.json')
m3u_filepaths_file = Path('playlists/m3u_playlist.m3u8')

# Load the data
def load_essentia_analysis():
    df = pd.read_json(essentia_descriptors)
    return df

audio_analysis = load_essentia_analysis()

audio_analysis.index = audio_analysis['file_id']
audio_analysis.head()

,file_id,bpm,danceability,voice_instru,valence-arousal,activations_dict
file_id,,,,,,
audio.000/1t/1tnci80rcW5LjqQqBwdZsW.mp3,audio.000/1t/1tnci80rcW5LjqQqBwdZsW.mp3,125.997940,4.005003,0.507735,5.576652,{'Blues---Boogie Woogie': 2.874757569770736e-0...
audio.000/5l/5lx1xGuMvgzCz9pn1eyjWN.mp3,audio.000/5l/5lx1xGuMvgzCz9pn1eyjWN.mp3,99.956627,1.753968,0.397437,5.101707,{'Blues---Boogie Woogie': 1.5599231346641307e-...
audio.000/3L/3LUPz3adlCwLL1Yl2IUHqL.mp3,audio.000/3L/3LUPz3adlCwLL1Yl2IUHqL.mp3,139.767349,1.242303,0.436567,6.524824,{'Blues---Boogie Woogie': 3.921186362276785e-0...
audio.000/3L/3L0Dg5V8V4XLugW7PXhKdk.mp3,audio.000/3L/3L0Dg5V8V4XLugW7PXhKdk.mp3,95.024933,1.130136,0.418706,5.058117,{'Blues---Boogie Woogie': 3.601374487516295e-0...
audio.000/3V/3VqHuw0wFlIHcIPWkhIbdQ.mp3,audio.000/3V/3VqHuw0wFlIHcIPWkhIbdQ.mp3,123.793266,1.100455,0.530349,4.764894,"{'Blues---Boogie Woogie': 0.000138510586111, '..."


In [2]:
# extract the activations dictionary column
activations = audio_analysis['activations_dict']

# create a new dataframe from the dictionary column
activations_df = pd.DataFrame.from_records(activations)

# concatenate the new activations dataframe with the original dataframe
audio_analysis = pd.concat([audio_analysis, activations_df], axis=1)

# drop the original activations dictionary column
audio_analysis = audio_analysis.drop('activations_dict', axis=1)


In [3]:
import numpy as np

# set the epsilon value
epsilon = 1e-8

# replace NaN and infinite values with 0
audio_analysis.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

# get the min and max values of each column in the range [5, 405]
x_min = audio_analysis.iloc[:, 5:405].min()
x_max = audio_analysis.iloc[:, 5:405].max()

# check if the range of a column is zero and replace it with a small non-zero value
zero_cols = np.where(x_min == x_max)[0]
for i in zero_cols:
    x_min[i] = x_min[i] - epsilon
    x_max[i] = x_max[i] + epsilon

# apply the normalization formula to each column
audio_analysis_norm = audio_analysis.iloc[:, 5:405].apply(lambda x: (x - x_min) / (x_max - x_min + epsilon))

# combine the normalized columns with the original columns
audio_analysis_norm = pd.concat([audio_analysis.iloc[:, :5], audio_analysis_norm], axis=1)

In [ ]:
audio_analysis_norm.head()